Fina Short UEP239 Final Project

In this project I will execute a suitability analysis determining the most suitable ZIP Code Tabulation Area (ZCTA) for recent college graduates to move to. The project focuses on the Boston Metropolitan Area as defined by the boundary of the Boston Region Metropolitan Planning Organization (MPO). The analysis will use four indicators to calculate suitability for this demographic, including 1) mean housing price/rent payment, 2) age distribution of population, 3) euclidean proximity to train node,

In [2]:
#Import dependencies
import rasterio
from rasterio.plot import show
from rasterio import features

import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

import richdem as rd
from scipy import ndimage
from rasterstats import zonal_stats

In [3]:
#Use the provided MA outline with detailed coastline OUTLINE25K_POLY.shp to clip the ZCTAs shapefile tl_2010_25_zcta510.shp into the more detailed outline. 
#This will remove the parts of the coastal ZCTAs that extend into the ocean, allowing for a better match with the Boston Region MPO boundary and making your maps more beautiful. 
#This can easily be done using the clip() function from GeoPandas.

#Extract the outline of the Boston Region MPO from the MPO boundaries shapefile. (This can be easily done using plain old Pandas boolean indexing.)
import numpy as np
import geopandas as gpd
#Extract all the ZCTAs whose centroid is within the Boston Region MPO boundary.

Data for Demographic & Social Indicators

In [ ]:
#indicator for age and sex, csv format
ACSST5Y2019.S0101

In [8]:
#read in bus stops file
buspoints= gpd.read_file("\\Users\\finas\\Documents\\my work\\School work\\Tufts GRAD\\Spring 2021\\UEP239\\uep239-final-project\\project_data\\MBTA_Bus_Stops\\MBTA_Bus_Stops.shp")
buspoints.head(8)

,OBJECTID,STOP_ID,STOP_NAME,TOWN,TOWN_ID,geometry
0,1,3077,Gallivan Blvd @ opp Marsh St,BOSTON,35,POINT (237120.669 892643.408)
1,2,841,Lagrange St @ Virgil Rd,BOSTON,35,POINT (227915.195 892644.017)
2,3,446,Norfolk St @ Nelson St,BOSTON,35,POINT (234385.661 892644.944)
3,4,847,Lagrange St opp Virgil St,BOSTON,35,POINT (227912.601 892650.156)
4,5,3079,Adams St @ Minot St,BOSTON,35,POINT (236644.812 892651.990)
5,6,8687,Adams St @ Minot St,BOSTON,35,POINT (236649.767 892672.199)
6,7,808,Centre St opp Bellevue st,BOSTON,35,POINT (228299.467 892674.321)
7,8,33076,Gallivan Blvd @ Hallet St,BOSTON,35,POINT (237367.877 892703.008)


In [9]:
#read in T stops file
tpoints= gpd.read_file("\\Users\\finas\\Documents\\my work\\School work\\Tufts GRAD\\Spring 2021\\UEP239\\uep239-final-project\\project_data\\MBTA_NODE\\MBTA_NODE.shp")
tpoints.head(8)

,STATION,LINE,TERMINUS,ROUTE,geometry
0,Ashmont,RED,Y,A - Ashmont C - Alewife,POINT (236007.538 892693.023)
1,Harvard,RED,N,A - Ashmont B - Braintree C - Alewife,POINT (231387.274 902684.016)
2,Kendall/MIT,RED,N,A - Ashmont B - Braintree C - Alewife,POINT (234087.917 901406.551)
3,Capen Street,RED,N,Mattapan Trolley,POINT (234055.438 890869.375)
4,Tufts Medical Center,ORANGE,N,Forest Hills to Oak Grove,POINT (235900.324 899934.313)
5,Waban,GREEN,N,D - Riverside,POINT (222197.281 897300.437)
6,Kenmore,GREEN,N,B C D,POINT (233324.505 899912.392)
7,Arlington,GREEN,N,B C D E,POINT (235346.130 900230.131)


#For each chosen indicator:
#If applicable, visualize any spatial data the indicator is based on. (For example, the locations of certain POIs.)

#Summarize the indicator values (like population density) for each ZCTA in the study area.


#Produce a choropleth map visualizing the indicator values and produce a ZCTA ranking based on the indicator values, reporting the highest and lowest ranking ZCTAs.



#Convert the indicator values into a suitability or vulnerability index or score. This can be done in two ways:


#Reclassify the indicator values based on classes of your choice into a five-point (or ten-point) score.

#Normalize the indicator values into a suitability or vulnerability index ranging from zero to one.